In [24]:
#making the csv files
import cv2
import numpy as np
import csv
from zipfile import ZipFile

label_map = {
    'bart_simpson': 0, 'charles_montgomery_burns': 1, 'homer_simpson': 2,
    'krusty_the_clown': 3, 'lisa_simpson': 4, 'marge_simpson': 5,
    'milhouse_van_houten': 6, 'moe_szyslak': 7, 'ned_flanders': 8,
    'principal_skinner': 9
}

def load_images_from_zip(zip_path, label_map):
    images, labels = [], []

    with ZipFile(zip_path, 'r') as zip_file:
        for file_info in zip_file.infolist():
            parts = file_info.filename.split('/')
            if len(parts) == 3:
                character_name, file_name = parts[1], parts[2]
                if character_name in label_map and file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    with zip_file.open(file_info) as file:
                        data = file.read()
                        if data:
                            image = cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_GRAYSCALE)
                            if image is not None:
                                image = cv2.resize(image, (28, 28))
                                image = (image * 255).astype(int)
                                images.append(image.flatten())
                                labels.append(label_map[character_name])

    return images, labels

base_path = 'train.zip'
output_csv_path = 'Character_TrainData.csv'

with open(output_csv_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['label'] + [f'pixel_{i}' for i in range(784)])

    images, labels = load_images_from_zip(base_path, label_map)

    for label, image in zip(labels, images):
        csv_writer.writerow([label] + list(image))

print(f"CSV file '{output_csv_path}' created successfully.")
base_path = 'train.zip'
output_csv_path = 'Character_TrainData.csv'


CSV file 'Character_TrainData.csv' created successfully.


In [25]:
#making the csv file for test
import cv2
import numpy as np
import csv
from zipfile import ZipFile

label_map = {
    'bart_simpson': 0, 'charles_montgomery_burns': 1, 'homer_simpson': 2,
    'krusty_the_clown': 3, 'lisa_simpson': 4, 'marge_simpson': 5,
    'milhouse_van_houten': 6, 'moe_szyslak': 7, 'ned_flanders': 8,
    'principal_skinner': 9
}

def load_images_from_zip(zip_path, label_map):
    images, labels = [], []

    with ZipFile(zip_path, 'r') as zip_file:
        for file_info in zip_file.infolist():
            parts = file_info.filename.split('/')
            if len(parts) == 3:
                character_name, file_name = parts[1], parts[2]
                if character_name in label_map and file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    with zip_file.open(file_info) as file:
                        data = file.read()
                        if data:
                            image = cv2.imdecode(np.frombuffer(data, np.uint8), cv2.IMREAD_GRAYSCALE)
                            if image is not None:
                                image = cv2.resize(image, (28, 28))
                                image = (image * 255).astype(int)
                                images.append(image.flatten())
                                labels.append(label_map[character_name])

    return images, labels

base_path = 'test.zip'
output_csv_path = 'Character_TestData.csv'

with open(output_csv_path, mode='w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['label'] + [f'pixel_{i}' for i in range(784)])

    images, labels = load_images_from_zip(base_path, label_map)

    for label, image in zip(labels, images):
        csv_writer.writerow([label] + list(image))

print(f"CSV file '{output_csv_path}' created successfully.")
base_path = 'train.zip'
output_csv_path = 'Character_TrainData.csv'


CSV file 'Character_TestData.csv' created successfully.


In [26]:
import pandas as pd
df = pd.read_csv("Character_TrainData.csv")
df = df[df["label"] != "label"]
df["label"] = df["label"].astype(int)
df.head()


,label,pixel_0,pixel_1,pixel_2,pixel_3,pixel_4,pixel_5,pixel_6,pixel_7,pixel_8,...,pixel_774,pixel_775,pixel_776,pixel_777,pixel_778,pixel_779,pixel_780,pixel_781,pixel_782,pixel_783
0,0,125,136,130,132,135,132,132,133,133,...,242,237,213,91,80,239,210,187,195,236
1,0,196,197,194,197,197,194,198,198,197,...,204,199,202,204,202,201,202,203,204,203
2,0,197,199,150,153,156,153,147,151,149,...,177,184,181,182,185,181,183,183,182,181
3,0,122,119,117,116,116,114,113,113,112,...,110,110,102,167,168,133,110,108,107,108
4,0,222,195,206,194,194,198,191,191,193,...,181,181,182,182,183,183,183,182,182,182


In [27]:
def relu(Z):
	return np.maximum(0, Z)

def relu_derivative(Z):
	return Z > 0

def softmax(Z):
	expZ = np.exp(Z - np.max(Z))
	return expZ / expZ.sum(axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
	m = y_true.shape[0]
	log_likelihood = -np.log(y_pred[range(m), y_true])
	loss = np.sum(log_likelihood) / m
	return loss

def one_hot(Y, C):
	Y = np.eye(C)[Y.reshape(-1)]
	return Y

In [ ]:
class SimpleCNN:
	def __init__(self, input_size, hidden_size, output_size):
		self.W1 = np.random.randn(input_size, hidden_size) * 0.01
		self.b1 = np.zeros((1, hidden_size))
		self.W2 = np.random.randn(hidden_size, hidden_size) * 0.01
		self.b2 = np.zeros((1, hidden_size))
		self.W3 = np.random.randn(hidden_size, output_size) * 0.01
		self.b3 = np.zeros((1, output_size))

	def forward(self, X):
		self.Z1 = np.dot(X, self.W1) + self.b1
		self.A1 = relu(self.Z1)
		self.Z2 = np.dot(self.A1, self.W2) + self.b2
		self.A2 = relu(self.Z2)
		self.Z3 = np.dot(self.A2, self.W3) + self.b3
		self.A3 = softmax(self.Z3)
		return self.A3

	def backward(self, X, Y, learning_rate=0.01):
		m = Y.shape[0]
		Y = one_hot(Y, self.A3.shape[1])

		dZ3 = self.A3 - Y
		dW3 = (1/m) * np.dot(self.A2.T, dZ3)
		db3 = (1/m) * np.sum(dZ3, axis=0, keepdims=True)

		dA2 = np.dot(dZ3, self.W3.T)
		dZ2 = dA2 * relu_derivative(self.Z2)
		dW2 = (1/m) * np.dot(self.A1.T, dZ2)
		db2 = (1/m) * np.sum(dZ2, axis=0, keepdims=True)

		dA1 = np.dot(dZ2, self.W2.T)
		dZ1 = dA1 * relu_derivative(self.Z1)
		dW1 = (1/m) * np.dot(X.T, dZ1)
		db1 = (1/m) * np.sum(dZ1, axis=0, keepdims=True)

		# Update weights and biases
		self.W1 -= learning_rate * dW1
		self.b1 -= learning_rate * db1
		self.W2 -= learning_rate * dW2
		self.b2 -= learning_rate * db2
		self.W3 -= learning_rate * dW3
		self.b3 -= learning_rate * db3

In [29]:
# Example usage
X_train = df.drop("label", axis=1).values
# Y = np.array([0, 1, 2, ..., 9])  # Example labels (for 10 classes)
Y_train = np.array(df["label"].values)

X_train = X_train+X_train.mean()/X_train.std()
learning_rate = 0.01
input_size = X_train.shape[1]
hidden_size = 100
output_size = 10
model = SimpleCNN(input_size, hidden_size, output_size)

predictions = model.forward(X_train)

# Compute loss
loss = cross_entropy_loss(predictions, Y_train)

# Backward pass and update weights
model.backward(X_train, Y_train, learning_rate)

In [ ]:
cost = []
epochs = 5000
learning_rate = 0.001
for i in range(epochs):
  if i%1000 == 0:
    print(f"{i},{loss}")
  predictions = model.forward(X_train)
  loss = cross_entropy_loss(predictions, Y_train)
  cost.append(loss)
  model.backward(X_train, Y_train, learning_rate)

0,2.329447190732549


In [ ]:
import matplotlib.pyplot as plt
plt.plot( np.linspace(0, epochs, epochs), np.array(cost))


In [21]:

print(min(cost))
df_test = pd.read_csv("Character_TestData.csv")
df_test = df[df["label"] != "label"]
df_test["label"] = df["label"].astype(int)
df_test.head()

In [ ]:
def predict(model, X):
	"""Make predictions with the model."""
	logits = model.forward(X)
	return softmax(logits)

def test_model(model, X_test, y_test):
	"""Test the model on the test dataset and return the accuracy."""
	predictions = predict(model, X_test)
	predicted_classes = np.argmax(predictions, axis=1)
	actual_classes = np.argmax(y_test, axis=1)
	accuracy = np.mean(predicted_classes == actual_classes)
	return accuracy

# Example usage:
# Assuming model is your trained SimpleCNN model
#X_test = np.random.randn(num_test_samples, 784)  # Example test input
X_test = df_test.drop("label", axis=1).values
#y_test = np.eye(10)[np.random.choice(10, num_test_samples)]  # Example test labels (one-hot encoded)
Y_test = np.eye(10)[np.array(df_test["label"].values)]

accuracy = test_model(model, X_test, Y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Cost after 0 iterations is : 325289.87795714743


<ipython-input-167-21dba92d0ff3>:8: RuntimeWarning: overflow encountered in exp
  expX = np.exp(x)
<ipython-input-167-21dba92d0ff3>:9: RuntimeWarning: invalid value encountered in divide
  return expX/np.sum(expX, axis = 0)


Cost after 10 iterations is : nan
Cost after 20 iterations is : nan
Cost after 30 iterations is : nan
Cost after 40 iterations is : nan
Cost after 50 iterations is : nan
Cost after 60 iterations is : nan
Cost after 70 iterations is : nan
Cost after 80 iterations is : nan
Cost after 90 iterations is : nan
